## <center style='color:purple'>PhytoScan: AI-Powered Plant Disease Detection</center>

Dataset is downloaded from: https://www.kaggle.com/datasets/tushar5harma/plant-village-dataset-updated/data

<h2 style="color:blue">Importing necessary libraries</h2>

In [4]:
import os
!pip install "numpy<2"
import numpy as np
!pip install tensorflow
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
import numpy as np
print(np.__version__)

1.26.4


In [6]:
import tensorflow as tf
print(tf.__version__)  # This should print the installed TensorFlow version

2.18.0


<h2 style="color:blue">Define dataset path</h2>

In [8]:
# Define dataset path
data_dir = "./plant-village-dataset" 
print(data_dir)

./plant-village-dataset


<h2 style="color:blue">Image parameters</h2>

In [10]:
# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

<h2 style="color:blue">Preprocessing Functions</h2>

In [12]:
from skimage import exposure

def preprocess_image(image):
    img_array = np.array(image) / 255.0  # Normalize first
    p2, p98 = np.percentile(img_array, (2, 98))
    return exposure.rescale_intensity(img_array, in_range=(p2, p98))

C:\Users\User\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


<h2 style="color:blue">Data Augmentation and Loading</h2>

In [14]:
# from skimage import exposure

# def contrast_stretching(image):
#     # Convert image to numpy array
#     img_array = np.array(image)
#     # Apply contrast stretching
#     p2, p98 = np.percentile(img_array, (2, 98))
#     img_rescaled = exposure.rescale_intensity(img_array, in_range=(p2, p98))
#     return img_rescaled


In [32]:
# Data Augmentation and Loading
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    # rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    # brightness_range=[0.8, 1.2],  # Adjust brightness
    # contrast_stretching=True,  # Enhance contrast
    # preprocessing_function=contrast_stretching,
    validation_split=0.2
)
# Load test set
test_datagen = ImageDataGenerator(rescale=1./255)

# # Data Augmentation and Loading
# datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2
# )

<h2 style="color:blue">Get all plant species folders</h2>

In [34]:
# Get all plant species folders
plant_species = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]

<h2 style="color:blue">Load Data for All Species</h2>

In [36]:
# Dictionary to store data generators per species
train_generators = {}
val_generators = {}
test_generators = {}

# Loop through each plant species folder
for plant in os.listdir(data_dir):
    plant_path = os.path.join(data_dir, plant)
    
    if not os.path.isdir(plant_path):
        continue  # Skip files if any

    print(f"Loading data for: {plant}")

    train_path = os.path.join(plant_path, "Train")
    val_path = os.path.join(plant_path, "Val")
    test_path = os.path.join(plant_path, "Test")

    # Train generator
    train_generators[plant] = datagen.flow_from_directory(
        train_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    # Validation generator
    val_generators[plant] = datagen.flow_from_directory(
        val_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    # Test generator
    test_generators[plant] = test_datagen.flow_from_directory(
        test_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )


Loading data for: Apple
Found 7771 images belonging to 4 classes.
Found 1747 images belonging to 4 classes.
Found 196 images belonging to 4 classes.
Loading data for: Bell Pepper
Found 3901 images belonging to 2 classes.
Found 877 images belonging to 2 classes.
Found 98 images belonging to 2 classes.
Loading data for: Cherry
Found 3509 images belonging to 2 classes.
Found 788 images belonging to 2 classes.
Found 89 images belonging to 2 classes.
Loading data for: Corn (Maize)
Found 7316 images belonging to 4 classes.
Found 1645 images belonging to 4 classes.
Found 188 images belonging to 4 classes.
Loading data for: Grape
Found 7222 images belonging to 4 classes.
Found 1623 images belonging to 4 classes.
Found 182 images belonging to 4 classes.
Loading data for: Potato
Found 5702 images belonging to 3 classes.
Found 1282 images belonging to 3 classes.
Found 144 images belonging to 3 classes.
Loading data for: Tomato
Found 11108 images belonging to 6 classes.
Found 2495 images belonging

<h2 style="color:blue">Load training and validation sets</h2>

# Load training and validation sets
train_generators = []
val_generators = []

for plant in plant_species:
    train_path = os.path.join(data_dir, plant, 'Train')
    val_path = os.path.join(data_dir, plant, 'Val')
    
    train_generator = datagen.flow_from_directory(
        train_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )
    val_generator = datagen.flow_from_directory(
        val_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )
    
    print(f"{plant}: Train - {train_generator.samples} images, {train_generator.num_classes} classes | "
          f"Validation - {val_generator.samples} images, {val_generator.num_classes} classes\n")
    
    train_generators.append(train_generator)
    val_generators.append(val_generator)



<h2 style="color:blue">Load test set</h2>

# Load test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generators = []

for plant in plant_species:
    test_path = os.path.join(data_dir, plant, 'Test')
    test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    print(f"{plant}: Test - {test_generator.samples} images, {test_generator.num_classes} classes\n")
    test_generators.append(test_generator)



<h2 style="color:blue">Define the model using EfficientNet</h2>

# Define the model using EfficientNet
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

<h2 style="color:blue">Freeze most of the model, but leave the last 20 layers trainable</h2>

# Freeze most of the model, but leave the last 20 layers trainable
for layer in base_model.layers[:-20]:  # Freeze all layers except the last 20
    layer.trainable = False

for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers
    layer.trainable = True

# # Freeze base model layers
# base_model.trainable = False

<h2 style="color:blue">Add custom classification layers</h2>

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(train_generator.num_classes, activation='softmax')(x)

<h2 style="color:blue">Create the model</h2>

# Create the model
model = Model(inputs=base_model.input, outputs=output_layer)

<h2 style="color:blue">Compile the model</h2>

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


<h2 style="color:blue">Train the model</h2>

# Train the model
history = model.fit(
    train_generators[0],
    validation_data=val_generators[0],
    epochs = 20,
    steps_per_epoch=len(train_generators[0]),
    validation_steps=len(val_generators[0])
)

<h2 style="color:blue">Train Separate Models per Plant</h2>

In [118]:
for plant in train_generators.keys():
    print(f"\nTraining model for: {plant}")

    num_classes = train_generators[plant].num_classes

    # Load EfficientNet model
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze last 20 layers for fine-tuning
    for layer in base_model.layers[-20:]:
        layer.trainable = True  

    # Add custom classification layers
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=output_layer)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        train_generators[plant],
        validation_data=val_generators[plant],
        epochs=20,
        steps_per_epoch=len(train_generators[plant]),
        validation_steps=len(val_generators[plant])
    )

    # Save model for each plant
    model.save(f"model_{plant}.h5")
    print(f"Model saved for {plant}!\n")



Training model for: Apple


C:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
243/243 ━━━━━━━━━━━━━━━━━━━━ 480s 2s/step - accuracy: 0.7771 - loss: 0.6164 - val_accuracy: 0.2267 - val_loss: 1.4287
Epoch 2/20
243/243 ━━━━━━━━━━━━━━━━━━━━ 431s 2s/step - accuracy: 0.9850 - loss: 0.0430 - val_accuracy: 0.3486 - val_loss: 1.5753
Epoch 3/20
243/243 ━━━━━━━━━━━━━━━━━━━━ 435s 2s/step - accuracy: 0.9930 - loss: 0.0208 - val_accuracy: 0.8140 - val_loss: 0.4669
Epoch 4/20
243/243 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.9957 - loss: 0.0157 - val_accuracy: 0.9267 - val_loss: 0.1924
Epoch 5/20
243/243 ━━━━━━━━━━━━━━━━━━━━ 435s 2s/step - accuracy: 0.9971 - loss: 0.0123 - val_accuracy: 0.8563 - val_loss: 0.3817
Epoch 6/20
243/243 ━━━━━━━━━━━━━━━━━━━━ 438s 2s/step - accuracy: 0.9954 - loss: 0.0141 - val_accuracy: 0.7876 - val_loss: 0.5569
Epoch 7/20
243/243 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.9965 - loss: 0.0122 - val_accuracy: 0.9908 - val_loss: 0.0244
Epoch 8/20
243/243 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.9980 - loss: 0.0054 - val_accu

Model saved for Apple!


Training model for: Bell Pepper
Epoch 1/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.8363 - loss: 0.3454 - val_accuracy: 0.5097 - val_loss: 0.6929
Epoch 2/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.9895 - loss: 0.0321 - val_accuracy: 0.5097 - val_loss: 0.8892
Epoch 3/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - accuracy: 0.9960 - loss: 0.0145 - val_accuracy: 0.5291 - val_loss: 1.4060
Epoch 4/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.9950 - loss: 0.0108 - val_accuracy: 0.5120 - val_loss: 1.7512
Epoch 5/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - accuracy: 0.9927 - loss: 0.0173 - val_accuracy: 0.6249 - val_loss: 1.2319
Epoch 6/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.9984 - loss: 0.0050 - val_accuracy: 0.7013 - val_loss: 1.0219
Epoch 7/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.9981 - loss: 0.0069 - val_accuracy: 0.6465 - val_loss: 1.0709
Epoch 8/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 

Model saved for Bell Pepper!


Training model for: Cherry
Epoch 1/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.8963 - loss: 0.2435 - val_accuracy: 0.4797 - val_loss: 0.7046
Epoch 2/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - accuracy: 0.9985 - loss: 0.0072 - val_accuracy: 0.2284 - val_loss: 0.7526
Epoch 3/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.9995 - loss: 0.0029 - val_accuracy: 0.4949 - val_loss: 0.7631
Epoch 4/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.9993 - loss: 0.0032 - val_accuracy: 0.5076 - val_loss: 0.6885
Epoch 5/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.9997 - loss: 0.0014 - val_accuracy: 0.9657 - val_loss: 0.1865
Epoch 6/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.9987 - loss: 0.0048 - val_accuracy: 0.8325 - val_loss: 0.3591
Epoch 7/20
110/110 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.9327 - val_loss: 0.1731
Epoch 8/20
110/110 ━━━━━━━━━━━━━━━━━━━━

Model saved for Cherry!


Training model for: Corn (Maize)
Epoch 1/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 446s 2s/step - accuracy: 0.8012 - loss: 0.5528 - val_accuracy: 0.1538 - val_loss: 1.5989
Epoch 2/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 401s 2s/step - accuracy: 0.9692 - loss: 0.0947 - val_accuracy: 0.2705 - val_loss: 1.8758
Epoch 3/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 400s 2s/step - accuracy: 0.9773 - loss: 0.0609 - val_accuracy: 0.5155 - val_loss: 1.5015
Epoch 4/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - accuracy: 0.9802 - loss: 0.0512 - val_accuracy: 0.6979 - val_loss: 0.8148
Epoch 5/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 400s 2s/step - accuracy: 0.9857 - loss: 0.0438 - val_accuracy: 0.4286 - val_loss: 1.6963
Epoch 6/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 400s 2s/step - accuracy: 0.9894 - loss: 0.0328 - val_accuracy: 0.5915 - val_loss: 1.3412
Epoch 7/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 401s 2s/step - accuracy: 0.9906 - loss: 0.0283 - val_accuracy: 0.8456 - val_loss: 0.4489
Epoch 8/20
229/229 ━━━━━━━━━━━━━━━━━━━

Model saved for Corn (Maize)!


Training model for: Grape
Epoch 1/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 445s 2s/step - accuracy: 0.7899 - loss: 0.5669 - val_accuracy: 0.2329 - val_loss: 1.3799
Epoch 2/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - accuracy: 0.9916 - loss: 0.0338 - val_accuracy: 0.6198 - val_loss: 0.9086
Epoch 3/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - accuracy: 0.9898 - loss: 0.0283 - val_accuracy: 0.9322 - val_loss: 0.1943
Epoch 4/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 403s 2s/step - accuracy: 0.9977 - loss: 0.0087 - val_accuracy: 0.9864 - val_loss: 0.0355
Epoch 5/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 400s 2s/step - accuracy: 0.9965 - loss: 0.0086 - val_accuracy: 0.7813 - val_loss: 0.5439
Epoch 6/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 402s 2s/step - accuracy: 0.9993 - loss: 0.0042 - val_accuracy: 0.8121 - val_loss: 0.4834
Epoch 7/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 402s 2s/step - accuracy: 0.9968 - loss: 0.0087 - val_accuracy: 0.6999 - val_loss: 1.1290
Epoch 8/20
226/226 ━━━━━━━━━━━━━━━━━━━━

Model saved for Grape!


Training model for: Potato
Epoch 1/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 352s 2s/step - accuracy: 0.8370 - loss: 0.4327 - val_accuracy: 0.3198 - val_loss: 1.0991
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 316s 2s/step - accuracy: 0.9943 - loss: 0.0201 - val_accuracy: 0.3237 - val_loss: 1.0881
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 316s 2s/step - accuracy: 0.9945 - loss: 0.0157 - val_accuracy: 0.3198 - val_loss: 1.1903
Epoch 4/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.9963 - loss: 0.0124 - val_accuracy: 0.9774 - val_loss: 0.0576
Epoch 5/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.9964 - loss: 0.0121 - val_accuracy: 0.8003 - val_loss: 0.4526
Epoch 6/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.9976 - loss: 0.0070 - val_accuracy: 0.9446 - val_loss: 0.1496
Epoch 7/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.9988 - loss: 0.0040 - val_accuracy: 0.8034 - val_loss: 0.5070
Epoch 8/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 316s 

Model saved for Potato!


Training model for: Tomato
Epoch 1/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 666s 2s/step - accuracy: 0.7164 - loss: 0.7965 - val_accuracy: 0.1667 - val_loss: 2.0973
Epoch 2/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 613s 2s/step - accuracy: 0.9569 - loss: 0.1141 - val_accuracy: 0.6024 - val_loss: 1.0896
Epoch 3/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 614s 2s/step - accuracy: 0.9803 - loss: 0.0602 - val_accuracy: 0.8309 - val_loss: 0.4867
Epoch 4/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 613s 2s/step - accuracy: 0.9851 - loss: 0.0436 - val_accuracy: 0.3964 - val_loss: 2.0198
Epoch 5/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 616s 2s/step - accuracy: 0.9873 - loss: 0.0351 - val_accuracy: 0.4754 - val_loss: 2.1939
Epoch 6/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 616s 2s/step - accuracy: 0.9906 - loss: 0.0317 - val_accuracy: 0.5331 - val_loss: 1.4339
Epoch 7/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 616s 2s/step - accuracy: 0.9915 - loss: 0.0275 - val_accuracy: 0.8882 - val_loss: 0.3219
Epoch 8/20
348/348 ━━━━━━━━━━━━━━━━━━━━ 617s

Model saved for Tomato!



<h2 style="color:blue">Evaluate each model</h2>

In [128]:
for plant in test_generators.keys():
    print(f"\nEvaluating model for: {plant}")

    # Load corresponding model
    model = tf.keras.models.load_model(f"model_{plant}.h5")

    # Evaluate on test set
    test_loss, test_acc = model.evaluate(test_generators[plant])
    print(f"Test Accuracy for {plant}: {test_acc:.4f}")

    # Generate classification report
    y_true = test_generators[plant].classes
    y_pred = np.argmax(model.predict(test_generators[plant]), axis=1)

    print(classification_report(y_true, y_pred, target_names=list(test_generators[plant].class_indices.keys())))



# # Evaluate the model
# test_loss, test_acc = model.evaluate(test_generators[0])
# print(f"Test Accuracy: {test_acc:.4f}")


Evaluating model for: Apple


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 297ms/step - accuracy: 0.9655 - loss: 0.0911
Test Accuracy for Apple: 0.9541
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 366ms/step
                  precision    recall  f1-score   support

      Apple Scab       0.91      0.98      0.94        51
       Black Rot       1.00      1.00      1.00        50
Cedar Apple Rust       1.00      0.82      0.90        44
         Healthy       0.93      1.00      0.96        51

        accuracy                           0.95       196
       macro avg       0.96      0.95      0.95       196
    weighted avg       0.96      0.95      0.95       196


Evaluating model for: Bell Pepper


C:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 244ms/step - accuracy: 1.0000 - loss: 0.0076
Test Accuracy for Bell Pepper: 1.0000


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 469ms/step
                precision    recall  f1-score   support

Bacterial Spot       1.00      1.00      1.00        48
       Healthy       1.00      1.00      1.00        50

      accuracy                           1.00        98
     macro avg       1.00      1.00      1.00        98
  weighted avg       1.00      1.00      1.00        98


Evaluating model for: Cherry


...

6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 320ms/step - accuracy: 0.9882 - loss: 0.0543
Test Accuracy for Grape: 0.9945
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 411ms/step
                      precision    recall  f1-score   support

           Black Rot       1.00      0.98      0.99        48
Esca (Black Measles)       1.00      1.00      1.00        48
             Healthy       0.98      1.00      0.99        43
         Leaf Blight       1.00      1.00      1.00        43

            accuracy                           0.99       182
           macro avg       0.99      0.99      0.99       182
        weighted avg       0.99      0.99      0.99       182


Evaluating model for: Potato


C:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 295ms/step - accuracy: 0.4083 - loss: 1.9498
Test Accuracy for Potato: 0.4514
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 433ms/step
              precision    recall  f1-score   support

Early Blight       1.00      0.14      0.25        49
     Healthy       0.41      1.00      0.58        46
 Late Blight       0.48      0.24      0.32        49

    accuracy                           0.45       144
   macro avg       0.63      0.46      0.39       144
weighted avg       0.63      0.45      0.38       144


Evaluating model for: Tomato


C:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 340ms/step - accuracy: 0.4875 - loss: 2.0101
Test Accuracy for Tomato: 0.5321
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 375ms/step
                        precision    recall  f1-score   support

        Bacterial Spot       0.87      0.63      0.73        43
          Early Blight       1.00      0.06      0.12        48
               Healthy       1.00      0.53      0.69        49
           Late Blight       0.27      1.00      0.43        47
    Septoria Leaf Spot       1.00      0.11      0.20        44
Yellow Leaf Curl Virus       1.00      0.84      0.91        49

              accuracy                           0.53       280
             macro avg       0.86      0.53      0.51       280
          weighted avg       0.86      0.53      0.52       280



<h2 style="color:blue">Confusion matrix</h2>

In [134]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize lists to store overall predictions and true labels
all_y_true = []
all_y_pred = []
all_class_labels = {}

# Loop through all species in test_generators
for i, species in enumerate(plant_species):
    test_generator = test_generators[i]
    
    # Get true labels and predictions
    y_true = test_generator.classes
    y_pred = np.argmax(model.predict(test_generator), axis=1)
    
    # Store results
    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)
    
    # Retrieve class labels for this species
    species_labels = list(test_generator.class_indices.keys())
    
    # Offset class indices for multi-species evaluation
    class_offset = len(all_class_labels)
    species_class_indices = {k: v + class_offset for k, v in test_generator.class_indices.items()}
    all_class_labels.update(species_class_indices)

# Reverse dictionary to get ordered class names
class_names = [k for k, v in sorted(all_class_labels.items(), key=lambda item: item[1])]

# Print classification report
print(classification_report(all_y_true, all_y_pred, target_names=class_names))

# Generate confusion matrix
cm = confusion_matrix(all_y_true, all_y_pred)

# Plot confusion matrix
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for Multi-Species Plant Disease Detection")
plt.show()



KeyError: 0

<h2 style="color:blue">Treatment Recommendation System</h2>

In [138]:
# Multi-species treatment recommendations
treatment_dict = {
    "Apple": {
        "Apple Scab": "Use fungicides like Captan or Mancozeb. Prune infected leaves.",
        "Black Rot": "Apply copper-based fungicides. Remove infected fruit.",
        "Cedar Apple Rust": "Use fungicides with myclobutanil. Plant resistant varieties.",
        "Healthy": "No treatment needed. Maintain proper watering and nutrients."
    },
    "Potato": {
        "Early Blight": "Apply chlorothalonil-based fungicides. Rotate crops.",
        "Late Blight": "Use systemic fungicides like Metalaxyl. Remove infected plants.",
        "Healthy": "No treatment needed. Maintain proper soil drainage."
    },
    "Bell Pepper": {
        "Bacterial Spot": "Use copper-based sprays and avoid overhead watering. Remove infected leaves.",
        "Healthy": "No treatment needed. Ensure proper air circulation."
    },
    "Cherry": {
        "Powdery Mildew": "Apply sulfur or potassium bicarbonate-based fungicides. Prune affected areas.",
        "Healthy": "No treatment needed. Avoid excessive humidity."
    },
    "Corn (Maize)": {
        "Cercospora Leaf Spot": "Apply strobilurin or triazole fungicides. Improve field sanitation.",
        "Common Rust": "Use resistant hybrids. Apply fungicides if severe.",
        "Northern Leaf Blight": "Apply fungicides with mancozeb or chlorothalonil. Rotate crops.",
        "Healthy": "No treatment needed. Ensure good soil health."
    },
    "Grape": {
        "Black Rot": "Apply fungicides like myclobutanil or mancozeb. Remove infected grapes.",
        "Esca (Black Measles)": "Use fungicides containing pyraclostrobin. Prune affected vines.",
        "Leaf Blight": "Apply copper-based fungicides. Improve vineyard ventilation.",
        "Healthy": "No treatment needed. Maintain balanced irrigation."
    },
    "Tomato": {
        "Bacterial Spot": "Use copper sprays. Avoid handling plants when wet.",
        "Early Blight": "Apply fungicides with chlorothalonil. Remove affected leaves.",
        "Late Blight": "Use systemic fungicides like Metalaxyl. Destroy infected plants.",
        "Septoria Leaf Spot": "Use fungicides with chlorothalonil. Space plants properly for airflow.",
        "Yellow Leaf Curl Virus": "Control whiteflies with insecticidal soap. Use resistant varieties.",
        "Healthy": "No treatment needed. Maintain proper fertilization."
    }
}

def recommend_treatment(plant, disease):
    if plant in treatment_dict and disease in treatment_dict[plant]:
        return treatment_dict[plant][disease]
    else:
        return "No specific treatment available. Consult an expert."

# Example usage
plant = "Grape"
disease = "Leaf Blight"
print(f"Recommended Treatment for {plant} - {disease}: {recommend_treatment(plant, disease)}")


Recommended Treatment for Grape - Leaf Blight: Apply copper-based fungicides. Improve vineyard ventilation.
